In [2]:
#install (only if necessary)
#library(devtools)
#install_github("nagdevAmruthnath/minio.s3")

# function to setup enviroment variables
setup_env <- function() {
    key <- Sys.getenv("JUPYTERLAB_S3_ACCESS_KEY_ID")
    secret <- Sys.getenv("JUPYTERLAB_S3_SECRET_ACCESS_KEY")
    endpoint <- Sys.getenv("JUPYTERLAB_S3_ENDPOINT")
    endpoint <- gsub("http://", "", endpoint)
    endpoint <- paste0(endpoint, ":80")
    Sys.setenv(
        "AWS_ACCESS_KEY_ID" = key,
        "AWS_SECRET_ACCESS_KEY" = secret,
        "AWS_DEFAULT_REGION" = "",
        "AWS_S3_ENDPOINT" = endpoint)
}

# function to put object with metadata in S3 storage
put_obj <- function(object, bucket, file, meta) {
    if (missing(meta))
        list()
    awsMeta <- list()
    for (i in seq(1,length(meta))){
      value <- meta[i]
      newMeta <- paste0('x-amz-meta-', names(value))
      awsMeta[newMeta] = value
    }
    r <- s3HTTP(
        verb = "PUT", 
        bucket = bucket,
        path = paste0('/', file),
        headers = awsMeta,
        request_body = object,
        write_disk = NULL,
        accelerate = FALSE,
        dualstack = FALSE,
        parse_response = TRUE, 
        check_region = FALSE,
        url_style = c("path", "virtual"),
        base_url = Sys.getenv("AWS_S3_ENDPOINT"),
        verbose = FALSE,
        region = "", 
        key = Sys.getenv("AWS_ACCESS_KEY_ID"), 
        secret = Sys.getenv("AWS_SECRET_ACCESS_KEY"), 
        session_token = NULL,
        use_https = FALSE)
    return(TRUE)
}

# function to put object tags
put_tags <- function(path, bucket, tags) {
    tags_char <- character(length(tags))
    for (i in seq_along(tags)) {
        tags_char[i] <- paste0("<Tag><Key>", names(tags)[i], "</Key><Value>", unname(tags[[i]]), "</Value></Tag>")
    }
    request_body <- paste0("<Tagging><TagSet>", paste0(tags_char, collapse = ""), "</TagSet></Tagging>")
    tmpfile <- tempfile()
    on.exit(unlink(tmpfile))
    cat(request_body, file = tmpfile)
    md <- base64enc::base64encode(digest::digest(file = tmpfile, raw = TRUE))
    r <- s3HTTP(
        verb = "PUT", 
        bucket = bucket,
        path = path,
        query = list(tagging = ""),
        request_body = tmpfile,
        headers = list(`Content-Length` = file.size(tmpfile), `Content-MD5` = md),
        write_disk = NULL,
        accelerate = FALSE,
        dualstack = FALSE,
        parse_response = TRUE, 
        check_region = FALSE,
        url_style = c("path", "virtual"),
        base_url = Sys.getenv("AWS_S3_ENDPOINT"),
        verbose = FALSE,
        region = Sys.getenv("AWS_DEFAULT_REGION"), 
        key = Sys.getenv("AWS_ACCESS_KEY_ID"), 
        secret = Sys.getenv("AWS_SECRET_ACCESS_KEY"), 
        session_token = NULL,
        use_https = FALSE)
    return(TRUE)
}

# function to get object metadata
get_meta <- function(path, bucket) {
    headers <- head_object(path, bucket, use_http = F)
    fields <- attributes(headers)
    meta <- list()
    for (i in seq(1,length(fields))){
        value <- fields[i]
        name <- names(value)
        if (startsWith(name, "x-amz-meta-"))
             meta[name] = value
    }
    return(meta)
}

# function to get object tags
get_tags <- function(path, bucket) {
    r <- s3HTTP(
        verb = "GET", 
        bucket = bucket,
        path = path,
        query = list(tagging = ""),
        headers = list(), 
        request_body = "",
        write_disk = NULL,
        accelerate = FALSE,
        dualstack = FALSE,
        parse_response = TRUE, 
        check_region = FALSE,
        url_style = c("path", "virtual"),
        base_url = Sys.getenv("AWS_S3_ENDPOINT"),
        verbose = FALSE,
        region = Sys.getenv("AWS_DEFAULT_REGION"), 
        key = Sys.getenv("AWS_ACCESS_KEY_ID"), 
        secret = Sys.getenv("AWS_SECRET_ACCESS_KEY"), 
        session_token = NULL,
        use_https = FALSE)
    return(r)
}

In [ ]:
setup_env()
library("minio.s3")
bucketlist(add_region = FALSE) # retrieve lists of buckets, check setup worked fine

bucketName <- 'XXX' **NOTE! use correct bucket name!**
fileName <- 'cars_test.csv'

# write an object with metadata in S3 storage
library("datasets")
tmp <- tempfile()
on.exit(unlink(tmp))
utils::write.csv(mtcars, file = tmp)
meta <- list("cat"="meow", "dog"="woof", "horse"="neigh")
put_obj(tmp, bucketName, fileName, meta)

# write object tags
tags <- list(color = "blue", size = "small")
put_tags(fileName, bucketName, tags)

In [4]:
# get object metadata and tags
get_meta(fileName, bucketName)
get_tags(fileName, bucketName)

# get an object from S3 storage and write it in local storage
raw <- get_object(fileName, bucketName, use_https = F)
txt <- rawToChar(raw)
write(txt, fileName)

$`x-amz-meta-cat`
[1] "meow"

$`x-amz-meta-dog`
[1] "woof"

$`x-amz-meta-horse`
[1] "neigh"

$Tag
$Tag$Key
[1] "color"

$Tag$Value
[1] "blue"


$Tag
$Tag$Key
[1] "size"

$Tag$Value
[1] "small"


attr(,"accept-ranges")
[1] "bytes"
attr(,"content-security-policy")
[1] "block-all-mixed-content"
attr(,"content-type")
[1] "application/xml"
attr(,"server")
[1] "envoy"
attr(,"strict-transport-security")
[1] "max-age=31536000; includeSubDomains"
attr(,"vary")
[1] "Origin,Accept-Encoding"
attr(,"x-amz-request-id")
[1] "1755CC9FE58F27B9"
attr(,"x-content-type-options")
[1] "nosniff"
attr(,"x-xss-protection")
[1] "1; mode=block"
attr(,"date")
[1] "Fri, 14 Apr 2023 12:23:43 GMT"
attr(,"x-envoy-upstream-service-time")
[1] "1"
attr(,"content-encoding")
[1] "gzip"
attr(,"transfer-encoding")
[1] "chunked"